In [78]:
import dgl
%matplotlib inline
import networkx as nx 
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import os
os.chdir('../graphwave/')

import matplotlib.pyplot as plt
import graphwave
from graphwave.shapes import build_graph

import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

width_basis = 200

### 1. Choose the basis (cycle, torus or chain)
basis_type = "cycle" 

### 2. Add the shapes 
n_shapes = 30  
list_shapes = [["house"]] * n_shapes + [["fan"]] * n_shapes + [["star"]] * n_shapes

### 3. Pass all these parameters to the Graph Structure
add_edges = 0 # random edges to add
G, communities, _ , role_id = build_graph.build_structure(width_basis, basis_type, list_shapes, start=0,
                                       add_random_edges=add_edges, plot=False,
                                       savefig=False)
d = dict(zip(np.unique(role_id), range(len(np.unique(role_id)))))
labels = np.array([d[i] for i in role_id])

In [79]:
G.number_of_nodes()

410

In [80]:
import torch
import torch.nn as nn
import dgl.function as fn
from dgl.nn.pytorch import edge_softmax, GATConv


class GAT(nn.Module):
    def __init__(self,
                 g,
                 num_layers,
                 in_dim,
                 num_hidden,
                 num_classes,
                 heads,
                 activation,
                 feat_drop,
                 attn_drop,
                 negative_slope,
                 residual):
        super(GAT, self).__init__()
        self.g = g
        self.num_layers = num_layers
        self.gat_layers = nn.ModuleList()
        self.activation = activation
        # input projection (no residual)
        self.gat_layers.append(GATConv(
            in_dim, num_hidden, heads[0],
            feat_drop, attn_drop, negative_slope, False, self.activation))
        # hidden layers
        for l in range(1, num_layers):
            # due to multi-head, the in_dim = num_hidden * num_heads
            self.gat_layers.append(GATConv(
                num_hidden * heads[l-1], num_hidden, heads[l],
                feat_drop, attn_drop, negative_slope, residual, self.activation))
        # output projection
        self.gat_layers.append(GATConv(
            num_hidden * heads[-2], num_classes, heads[-1],
            feat_drop, attn_drop, negative_slope, residual, None))

    def forward(self, inputs):
        h = self.g.in_degrees().view(-1, 1).float()
        #h = inputs
        for l in range(self.num_layers):
            h = self.gat_layers[l](self.g, h).flatten(1)
        # output projection
        logits = self.gat_layers[-1](self.g, h).mean(1)
        return logits

In [81]:
import torch
labels = torch.tensor(labels)
inputs = torch.eye(G.number_of_nodes())

In [82]:
S = dgl.DGLGraph()
S.from_networkx(G)
S.ndata['h'] = inputs

In [83]:
inputs.shape

torch.Size([410, 410])

In [84]:
np.random.seed(10)
labeled_nodes = np.random.choice(list(range(G.number_of_nodes())), 100, replace = False)
labels_train = labels[labeled_nodes]
unlabelled_nodes = [i for i in list(range(G.number_of_nodes())) if i not in labeled_nodes]
test_label = labels[unlabelled_nodes]

In [85]:
labeled_nodes

array([ 81, 247,   1, 170, 341, 274, 376,  65, 386, 149, 189, 164, 121,
       293, 242, 152,  66, 151,  43, 384, 166, 192, 260,  88, 361, 305,
        47, 323,  97, 298,  24, 100,  99, 288, 102, 126, 191, 400, 278,
       241,   6, 147,  26, 362, 211,  56, 358, 101,  61, 276, 367, 389,
        72,  37, 330, 183, 281, 207, 187,  12, 350, 223, 228, 105, 387,
       110, 236, 259,  98, 222, 277, 258, 370, 266,  78, 113, 295, 316,
       385, 163,  52, 372, 217, 227,  64,  27, 282, 300, 111, 297, 328,
       109, 231, 208, 119, 275, 395,  91,  69, 352])

In [86]:
heads = ([2] * 2) + [1]
net = GAT(S, 2, 1, 4, 10, heads, F.elu, .3, .3, .2, False)

optimizer = torch.optim.Adam(net.parameters(), lr=0.005)
all_logits = []
for epoch in range(2000):
    logits = net(inputs)
    # we save the logits for visualization later
    logp = F.log_softmax(logits, 1)
    # we only compute loss for labeled nodes
    loss = F.nll_loss(logp[labeled_nodes], labels_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch%100 == 0:
        print('Epoch %d | Loss: %.4f' % (epoch, loss.item()))


Epoch 0 | Loss: 4.6098
Epoch 100 | Loss: 2.1906
Epoch 200 | Loss: 2.1331
Epoch 300 | Loss: 2.1165
Epoch 400 | Loss: 2.1686
Epoch 500 | Loss: 2.1144
Epoch 600 | Loss: 2.1279
Epoch 700 | Loss: 2.1324
Epoch 800 | Loss: 2.1117
Epoch 900 | Loss: 2.1806
Epoch 1000 | Loss: 2.0348
Epoch 1100 | Loss: 2.1845
Epoch 1200 | Loss: 2.0105
Epoch 1300 | Loss: 1.9812
Epoch 1400 | Loss: 2.0029
Epoch 1500 | Loss: 1.8286
Epoch 1600 | Loss: 1.9952
Epoch 1700 | Loss: 1.8913
Epoch 1800 | Loss: 1.7601
Epoch 1900 | Loss: 1.9091


In [87]:
net.eval()
logits = net(inputs)
# we save the logits for visualization later
logp = F.log_softmax(logits, 1)
# we only compute loss for labeled nodes
#loss = F.nll_loss(logp[labeled_nodes], labels_train)

In [88]:
logp

tensor([[-2.7576, -2.3265, -4.0423,  ..., -4.0592, -1.7680, -1.9885],
        [-2.4467, -2.0291, -4.4223,  ..., -4.4248, -1.7731, -1.9679],
        [-2.8504, -2.4166, -3.9592,  ..., -3.9801, -1.7683, -1.9947],
        ...,
        [-2.4955, -2.0899, -4.5525,  ..., -4.5579, -1.7314, -1.9173],
        [-2.4955, -2.0899, -4.5525,  ..., -4.5579, -1.7314, -1.9173],
        [-2.4422, -2.0318, -4.5100,  ..., -4.5125, -1.7563, -1.9450]],
       grad_fn=<LogSoftmaxBackward>)

In [89]:
argmax_Y = torch.max(logp[unlabelled_nodes], 1)[1]

In [90]:
print('Accuracy of argmax predictions on the test set: {:4f}%'.format(
    (test_label == argmax_Y.float()).sum().item() / len(test_label) * 100))

Accuracy of argmax predictions on the test set: 26.451613%
